In [9]:
import math
import time
import copy
from datetime import datetime
from pathlib import Path
import shutil
from glob import glob

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F

from PIL import Image
from tqdm.auto import tqdm

import pandas as pd
import numpy as np

METRIC_DIM = 512
IMG_SIZE = 112
device = torch.device("cuda:0")

In [10]:
# 훈련 이미지 리스트
BASE_DIR = '/data/kts123/aihub/reid'
train_imgs = '/data/kts123/aihub/reid/img_list_train.txt'

In [ ]:
df = pd.read_csv(train_imgs)
for kls, df_i in tqdm(df.groupby('KLS_IDX')):
    dst_dir = f'train_imgs/{kls:05d}'
    Path(dst_dir).mkdir(exist_ok=True, parents=True)
    for name in df_i['NAME'].values:
        src = f'{BASE_DIR}/{name}'
        dst = f'{dst_dir}/{Path(name).name}'
        shutil.copy(src, dst)

In [14]:
#val 이미지 리스트
val_imgs = '/data/kts123/aihub/reid/img_list_val.txt'
df = pd.read_csv(train_imgs)

In [ ]:
for kls, df_i in tqdm(df.groupby('KLS_IDX')):
    dst_dir = f'val_imgs/{kls:05d}'
    Path(dst_dir).mkdir(exist_ok=True, parents=True)
    for name in df_i['NAME'].values:
        src = f'{BASE_DIR}/{name}'
        dst = f'{dst_dir}/{Path(name).name}'
        shutil.copy(src, dst)

  0%|          | 0/502 [00:00<?, ?it/s]

In [1]:
from glob import glob
from torch.utils.data import Dataset, DataLoader

class FolderDataset(Dataset):
    def __init__(self, root_dir, phase = 'train', trsf = lambda e: e):
        imgs_total = glob(f'{root_dir}/*/*.jpg')
        imgs_train = [e for e in imgs_total if int(Path(e).stem) < 100]
        imgs_val   = [e for e in imgs_total if int(Path(e).stem) >= 100]
        self.imgs = imgs_train if phase == 'train' else imgs_val
        
        self.class_names = set([p.split('/')[-2] for p in self.imgs])
        self.class_names = sorted(list(self.class_names))
        self.labels = [self.class_names.index(e.split('/')[-2]) for e in self.imgs]
        self.trsf = trsf
        
    def num_labels(self):
        return len(self.class_names)